In [ ]:
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
from glob import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGESHAPE = [224, 224, 3] 

vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)

for each_layer in vgg_model.layers:
    each_layer.trainable = False

classes = glob('chest_xray/chest_xray/train')

flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(2, activation='softmax')(flatten_layer)

final_model = Model(inputs=vgg_model.input, outputs=prediction)
final_model.summary()

final_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

testing_datagen = ImageDataGenerator(rescale=1./255)


training_set = train_datagen.flow_from_directory(
    'chest_xray/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
test_set = testing_datagen.flow_from_directory(
    'chest_xray/test',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)
val_set = testing_datagen.flow_from_directory(
    'chest_xray/val',
    target_size=(224, 224),
    batch_size=4,
    class_mode='categorical'
)

In [ ]:
fitted_model = final_model.fit(
    training_set,
    validation_data=test_set,
    epochs=1)

In [ ]:
final_model.save('final_vgg16_3.keras')

In [ ]:
from keras.preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = load_model('final_vgg16_3.keras')
image_path = 'chest_xray/val/PNEUMONIA/person1952_bacteria_4883.jpeg'
target_size = (224, 224)

loaded_image = image.load_img(image_path, target_size=target_size)
image_array = image.img_to_array(loaded_image)
image_array_expanded = np.expand_dims(image_array, axis=0)
processed_image_data = preprocess_input(image_array_expanded)
prediction = model.predict(processed_image_data)

if prediction[0][0] > prediction[0][1]:
    print('Person is safe.')
else:
    print('Person is affected with Pneumonia.')

print(f'Predictions: {prediction}')


In [ ]:
loss, accuracy = final_model.evaluate(val_set)
print('The accuracy of the model on test dataset is',
      np.round(accuracy*100))

In [ ]:
plot.plot(fitted_model.history['loss'], label='training loss')
plot.plot(fitted_model.history['val_loss'], label='validation loss')
plot.legend()
plot.savefig('LossVal_loss')
plot.show()

plot.plot(fitted_model.history['accuracy'], label='training accuracy')
plot.plot(fitted_model.history['val_accuracy'], label='validation accuracy')
plot.legend()
plot.savefig('AccVal_acc')
plot.show()

In [ ]:
# Train the model
fitted_model = final_model.fit(
    training_set,
    validation_data=test_set,  # Use val_set for validation
    epochs=5  # Increased number of epochs
)


In [ ]:

# Save the model
final_model.save('final_vgg16_2.keras')

# Load and preprocess an image for prediction
from keras.preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np

# Load the pre-trained model
model = load_model('final_vgg16_2.keras')

# Load and preprocess the image
img = image.load_img('chest_xray/val/PNEUMONIA/person1950_bacteria_4881.jpeg', target_size=(224, 224))
image_array = image.img_to_array(img)
image_array = np.expand_dims(image_array, axis=0)
img_data = preprocess_input(image_array)

# Make a prediction
prediction = model.predict(img_data)

# Interpret the prediction
class_index = np.argmax(prediction, axis=1)[0]  # Get predicted class index
if class_index == 0:
    print('Person is safe.')
else:
    print('Person is affected with Pneumonia.')

print(f'Predictions: {prediction}')

# Evaluate the model
loss, accuracy = final_model.evaluate(val_set)
print('The accuracy of the model on validation dataset is', np.round(accuracy * 100, 2))

# Plotting training and validation loss
plot.figure()
plot.plot(fitted_model.history['loss'], label='Training Loss')
plot.plot(fitted_model.history['val_loss'], label='Validation Loss')
plot.title('Training and Validation Loss')
plot.xlabel('Epochs')
plot.ylabel('Loss')
plot.legend()
plot.grid()
plot.savefig('LossVal_loss')  # Save before showing
plot.show()

# Plotting training and validation accuracy
plot.figure()
plot.plot(fitted_model.history['accuracy'], label='Training Accuracy')
plot.plot(fitted_model.history['val_accuracy'], label='Validation Accuracy')
plot.title('Training and Validation Accuracy')
plot.xlabel('Epochs')
plot.ylabel('Accuracy')
plot.legend()
plot.grid()
plot.savefig('AccVal_acc')  # Save before showing
plot.show()